In [1]:
from tkinter import filedialog
import tkinter as tk
import cv2
import PIL.Image, PIL.ImageTk
import numpy as np
from PIL import ImageTk, Image 

# global variables
MAXDIM = 400

class App():
    def __init__(self, window, window_title, image_path="rose.jpeg"):
        self.window = window
        self.window.title(window_title)
        
        # Load an image using OpenCV
        self.cv_img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        self.NEWcv_img = self.cv_img.copy()  # for recursive processing
        
        # Get the image dimensions (OpenCV stores image data as NumPy ndarray)
        self.height, self.width, no_channels = self.cv_img.shape
        
        ''' Image Display Related Code'''
        # Create a FRAME that can fit the images
        self.frame1 = tk.Frame(self.window, width=self.width, height=self.height)
        self.frame1.pack(fill=tk.BOTH)
        
        # Create a CANVAS for original image
        self.canvas0 = tk.Canvas(self.frame1, width=MAXDIM, height=MAXDIM)
        self.canvas0.pack(side=tk.LEFT, expand=1, fill=tk.BOTH)
        
        # Create a CANVAS for changing image
        self.canvas1 = tk.Canvas(self.frame1, width=MAXDIM, height=MAXDIM)
        self.canvas1.pack(side=tk.RIGHT, expand=1, fill=tk.BOTH)
        
        # Use PIL (Pillow) to convert the NumPy ndarray to a PhotoImage
        self.photoOG = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(self.cv_img))
        self.photo = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(self.cv_img))
        
        # Add a PhotoImage to the Canvas (original)
        self.canvas0.create_image(MAXDIM//2, MAXDIM//2, image=self.photoOG)
        
        # Add a PhotoImage to the Canvas (changing effects)
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
# ##############################################################################################
# ################################   PARAMETER TOOLBAR   #######################################
# ##############################################################################################
        
        # Create a FRAME that can fit the labels
        self.frame2 = tk.Frame(self.window, width=self.width, height=50)
        self.frame2.pack(expand=1, fill=tk.BOTH)
        
        # Create a CANVAS for the original image label
        self.canvas2 = tk.Canvas(self.frame2, width=MAXDIM, height=50)
        self.canvas2.pack(side=tk.LEFT, expand=1, fill=tk.BOTH)
        
        # Create a CANVAS for the modified image label
        self.canvas3 = tk.Canvas(self.frame2, width=MAXDIM, height=50)
        self.canvas3.pack(side=tk.RIGHT, expand=1, fill=tk.BOTH)
        
        # Write labels for both images, font/size can be changed
        self.canvas2.create_text(MAXDIM//2, 25,font="Tahoma 20",text="Original Photo")
        self.canvas3.create_text(MAXDIM//2, 25,font="Tahoma 20",text="Modified Photo")
    
        # Create a FRAME that can fit the features
        self.frame3 = tk.Frame(self.window, width=self.width, height=self.height, bg='green')
        self.frame3.pack(side=tk.BOTTOM, expand=1, fill=tk.BOTH)
        
        # Create a BUTTON that resets the image
        self.btn_reset = tk.Button(self.frame3, text="Reset Image", font="Tahoma 10", command=self.reset)
        self.btn_reset.pack(side=tk.LEFT, padx = 20)
        
        # Create a BUTTON that loads a new image
        self.btn_load = tk.Button(self.frame3, text="Load New Image", font="Tahoma 10", command=self.load)
        self.btn_load.pack(side=tk.RIGHT, padx = 20)
        
        # Create a SCALE that lets the user control the level of cartoonization
        self.scl_car = tk.Scale(self.frame3, from_=0, to=150, orient=tk.HORIZONTAL, showvalue=1,
                command = self.cartoonize, length=150, sliderlength=25, tickinterval=150, label="  Set Sigma Value", font="Tahoma 12")
        self.scl_car.pack(side=tk.BOTTOM, pady = 30)
        
        self.window.mainloop()

##############################################################################################
#################################  CALLBACK FUNCTIONS  #######################################
##############################################################################################

    # Callback for reset button
    def reset(self):        
        self.NEWcv_img = self.cv_img
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.NEWcv_img))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        self.scl_car.set(0)
        
    # Callback for load new image button
    def load(self):
        
        filename = filedialog.askopenfilename() 
        img = cv2.imread(filename)
        self.cv_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.photoOG = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(self.cv_img))
        self.canvas0.create_image(MAXDIM//2, MAXDIM//2, image=self.photoOG)
        self.reset()
    
    # Callback for cartoonization scale
    def cartoonize(self, sigma_value):
        sigma_value = self.scl_car.get()
        if (sigma_value == 0):
            self.NEWcv_img = self.cv_img
        else:
            c_value = int(-0.08*sigma_value + 15)
            bil_filter = cv2.bilateralFilter(self.cv_img, 9, sigma_value, sigma_value)
            cv_img_grayscale = cv2.cvtColor(self.cv_img, cv2.COLOR_RGB2GRAY)  
            cv_img_grayscale_blurred = cv2.medianBlur(cv_img_grayscale, 5)  
            cv_img_grayscale_edges = cv2.adaptiveThreshold(cv_img_grayscale_blurred, 255,  
                                            cv2.ADAPTIVE_THRESH_MEAN_C,  
                                            cv2.THRESH_BINARY, 5, c_value)  
            cv_img_color_edges = cv2.cvtColor(cv_img_grayscale_edges, cv2.COLOR_GRAY2RGB)
            self.NEWcv_img = cv2.bitwise_and(bil_filter, cv_img_color_edges)
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.NEWcv_img))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        

##############################################################################################
# Create a window and pass it to the Application object
App(tk.Tk(), "Sample Final Project, Simple")     